In [1]:
import pandas as pd
import numpy as np
import shutil
from IPython.display import display
from rdkit import Chem
from rdkit.Chem import rdFMCS, AllChem, Draw
from morfeus.sasa import SASA
from morfeus.buried_volume import BuriedVolume
from morfeus.conformer import ConformerEnsemble
from morfeus.xtb import XTB
from morfeus.dispersion import Dispersion
from tqdm import tqdm
import os
from pathlib import Path

Define Functions for the calculation of the descriptors

In [2]:
def append_occurrence_numbers(strings):
    counts = {}
    result = []
    for s in strings:
        counts[s] = counts.get(s, 0) + 1
        result.append(f"{s}_{counts[s]}")
    return result

def generate_template(ref_mol,core_mol):

    match = ref_mol.GetSubstructMatch(core_mol)

    # Create a substructure mol (just the MCS) with atom mapping
    em = Chem.EditableMol(Chem.Mol())
    atom_map = {}
    for new_idx, old_idx in enumerate(match):
        atom = ref_mol.GetAtomWithIdx(old_idx)
        atom_map[old_idx] = new_idx
        new_atom = Chem.Atom(atom.GetAtomicNum())
        new_atom.SetAtomMapNum(new_idx + 1)  # Atom map numbers start at 1
        em.AddAtom(new_atom)

    # Add bonds within the MCS
    for i, idx1 in enumerate(match):
        for j, idx2 in enumerate(match):
            if idx1 >= idx2:
                continue
            bond = ref_mol.GetBondBetweenAtoms(idx1, idx2)
            if bond:
                em.AddBond(atom_map[idx1], atom_map[idx2], bond.GetBondType())

    mcs_submol = em.GetMol()
    smarts_with_map = Chem.MolToSmarts(mcs_submol)
    template = Chem.MolFromSmarts(smarts_with_map)

    return template

def get_mcs_template_with_consistent_atom_order(smiles_list):
    # Convert SMILES to H-added mols
    mols = [Chem.AddHs(Chem.MolFromSmiles(smi)) for smi in smiles_list]

    # Find MCS
    mcs_result = rdFMCS.FindMCS(mols)
    mcs_mol = Chem.RemoveAllHs(Chem.MolFromSmarts(mcs_result.smartsString))

    # Use first molecule as reference for the template generation
    template = generate_template(ref_mol=mols[0],core_mol=mcs_mol)
    return template


def map_common_core_with_consistent_atom_order(smiles_list,common_core):
    # generate mol objects for smiles_list
    mols = [Chem.AddHs(Chem.MolFromSmiles(smiles)) for smiles in smiles_list]

    # generate mol for common_core
    core_mol = Chem.RemoveAllHs(Chem.MolFromSmarts(common_core))

    # Use first molecule as reference for the template generation
    template = generate_template(ref_mol=mols[0],core_mol=core_mol)
    return template

In [3]:
def calculate_morfeus_descriptors (smiles_list,filename,common_core=None,chunk_size=25,starting_smiles=1,chunk_label=1):
    """
    Calculate morfeus descriptors at the GFN2-xTB level for  a list of given smiles strings.
    ---------------
    smiles_list: list
        list of smiles strings
    common_core: str or None
        smarts for the common core of interest
        Default is None --> will look for the largest common substructure in the molecule
    filename: str
        path for the generated dataset
    common_core: str or None
        SMARTS for a substructure for which atom descriptors will be extracted
        If common_core=None (Default), the atom descriptors will be calculated for the
        maximum common substructure.
    chunk_size: int
        number of compounds that will be calculated in one chunk before saving the obtained data
        at the end of the run, all chunks will be concatenated.
        Default: 25
    starting_smiles: int (one-indexed)
        first entry of the smiles list to be calculated
        Default: 1
        (useful for restarting in case the calculation crashes)
    chunk_label: int
        label for the next chunk to be calculated
        Default: 1
        (useful for restarting in case the calculation crashes)
    ----------------------
    Save the descriptor data under the indicated filename
    Also returns a list of pd.DataFrames for the data of the individual chunks
    as well as a pd.DataFrame for the combined Data of all calculated structures

    """

    print("This might take several minutes or even hours. Please stand by.")
    # Find common substructure and align the template
    # Create a folder to save the separate chunks of calculation results
    if not os.path.exists("./morfeus_chunks"):
        # Create the folder
        os.makedirs("./morfeus_chunks")
    template = None
    if common_core is None:
        template = get_mcs_template_with_consistent_atom_order(smiles_list)
    else:
        template = map_common_core_with_consistent_atom_order(smiles_list,common_core)
    print("Atom properties will be calculated for the following common substructure:")
    depiction = Draw.MolToImage(template)
    display(depiction)
    pt = Chem.GetPeriodicTable()

    results = []
    smiles_list_chunk = []
    properties = None

    for smiles_index, smiles in enumerate(tqdm(smiles_list[(starting_smiles-1):])):
        smiles_index = smiles_index + starting_smiles -1
        current_results = {}
        # Generate conformer ensemble
        ce = ConformerEnsemble.from_rdkit(smiles,optimize="MMFF94")
        ce.prune_rmsd()
        ce.sort()
        if len(ce) > 5:
            ce = ce[:5]  # prune to top 5 conformers 
        # Optimize conformers
        model = {"method": "GFN2-xTB"}
        ce.optimize_qc_engine(program="xtb",model=model,procedure="geometric")
        ce.sp_qc_engine(program="xtb",model=model)
        ce.prune_energy()
        # Get the matching substructure (excluding hydrogens)
        match = ce.mol.GetSubstructMatch(template)
        substruct_atoms = [pt.GetElementSymbol(int(ce.elements[nr])) for nr in match]
        substruct_labels = append_occurrence_numbers(substruct_atoms)

        for conformer in ce:
            props = conformer.properties
            sasa = SASA(ce.elements, conformer.coordinates)
            disp = Dispersion(ce.elements, conformer.coordinates)
            xtb = XTB(ce.elements, conformer.coordinates)

            props["SASA"] = sasa.area
            props["Volume"] = disp.volume
            props["HOMO"] = xtb.get_homo()
            props["LUMO"] = xtb.get_lumo()
            props["IP"] = xtb.get_ip(corrected=True)
            props["EA"] = xtb.get_ea(corrected=True)
            props["Dipole"] = np.linalg.norm(xtb.get_dipole())

            sasa_atom_areas = sasa.atom_areas
            disp_atom_p_int = disp.atom_p_int
            charges = xtb.get_charges()
            electrophilicity = xtb.get_fukui("electrophilicity")
            nucleophilicity = xtb.get_fukui("nucleophilicity")
            radical_fukui = xtb.get_fukui("radical")

            for i,idx in enumerate(match):
                atom_label = substruct_labels[i]
                bv = BuriedVolume(ce.elements, conformer.coordinates, idx+1)
                props[f"{atom_label}_BV"] = bv.fraction_buried_volume
                props[f"{atom_label}_SASA"] = sasa_atom_areas[idx+1]
                props[f"{atom_label}_P_int"] = disp_atom_p_int[idx+1]
                props[f"{atom_label}_charge"] = charges[idx+1]
                props[f"{atom_label}_electrophilicity"] = electrophilicity[idx+1]
                props[f"{atom_label}_nucleophilicity"] = nucleophilicity[idx+1]
                props[f"{atom_label}_radicalFukui"] = radical_fukui[idx+1]
            if smiles == smiles_list[(starting_smiles-1)] and conformer == ce[0]:
                properties = props.keys()

        for property in properties:
            current_results[property] = ce.boltzmann_statistic(property)
        results.append(current_results)

        smiles_list_chunk.append(smiles)  # add the smiles to the list of smiles in this chunk

        if (smiles_index + 1) % chunk_size == 0:  # check if the chunk is full; +1 due to zero-indexing
            pd.DataFrame(results,index=smiles_list_chunk,columns=properties).to_csv(f"./morfeus_chunks/morfeus_chunk_{chunk_label}.csv",index=True,header=True)
            # clean out the collection variables for the next chunk
            results = []
            smiles_list_chunk = []
            chunk_label += 1  # update the chunk label for the next chunk
    
    # once all smiles have been calculated, save the last samples in a final chunk
    if smiles_list_chunk:
        pd.DataFrame(results,index=smiles_list_chunk,columns=properties).to_csv(f"./morfeus_chunks/morfeus_chunk_{chunk_label}.csv",index=True,header=True)


    # combine the chunks and clean up
    dfs = []
    for chunk_file in os.listdir("./morfeus_chunks"):
        dfs.append(pd.read_csv(f"./morfeus_chunks/{chunk_file}",index_col=0,header=0))
    df_combined = pd.concat(dfs,axis=0)
    df_combined.to_csv(filename,index=True,header=True)
    shutil.rmtree("./morfeus_chunks")
    os.remove("qce_optim.xyz")

    return dfs, df_combined

Calculate descriptors for the acids

In [ ]:
# # Load the SMILES strings of the dataset.
# smiles_acids = pd.read_csv("amide_smiles_substrates_acids.csv",index_col=0,header=0).index.to_list()
# print(f"Read in {len(smiles_acids)} SMILES strings.")
# # calculate descriptors
# _,_ = calculate_morfeus_descriptors (smiles_list = smiles_acids,common_core = None,
#                                      filename = "./../1_Dataset_Generation/Data_For_Individual_Substrates/"\
#                                         "amide_morfeus_descr_acids.csv")

Calculate descriptors for the amines

In [ ]:
# # Load the SMILES strings of the dataset.
# smiles_amines = pd.read_csv("amide_smiles_substrates_amines.csv", index_col = 0, header = 0).index.to_list()
# print(f"Read in {len(smiles_amines)} SMILES strings.")
# # calculate descriptors
# _,_ = calculate_morfeus_descriptors (smiles_list = smiles_amines, common_core = "[NH2,NH]", 
#                                      filename="./../1_Dataset_Generation/Data_For_Individual_Substrates/"\
#                                         "amide_morfeus_descr_amine.csv")

Calculate descriptors for the amide products

In [ ]:
# # Load the SMILES strings of the dataset.
# smiles_amides = pd.read_csv("amide_smiles_products.csv",index_col=0,header=0).index.to_list()
# print(f"Read in {len(smiles_amides)} SMILES strings.")
# # calculate descriptors (Sometimes the function stops running. In these cases, it can be restarted for the rest of the data by changing starting_smiles and chunk_label)
# _,_ = calculate_morfeus_descriptors (smiles_list=smiles_amides,common_core="[C,c]C(=O)N",
#                                filename="./../1_Dataset_Generation/Data_For_Individual_Substrates/"\
#                                 "amide_morfeus_descr_prods.csv",
#                                 chunk_label=26,starting_smiles=626)